# Cassava Leaf Disease Classification – Plan

Goal: Win a medal (>= 0.8978 accuracy).

Data available:
- train_images/ (21k imgs), train.csv with labels 0..4
- test_images/ (~3k imgs) to predict
- label map json
- TFRecords exist but we can start from raw JPEGs for simplicity; may switch if throughput needed

Metric: Accuracy (LB).

High-level approach:
1) Establish strong, fast baseline using pretrained ImageNet models from timm on GPU with mixed precision.
   - Start with convnext_base or efficientnet_b3/b4, image_size=512 (or 380/448 first for speed).
   - Augmentations: flips, light affine, color jitter, CutMix/MixUp (prob ~0.2), RandomResizedCrop, Normalize.
   - Loss: cross-entropy with label smoothing (0.05) or focal for class imbalance; track which wins on CV.
   - Optimizer: AdamW, lr ~2e-4, cosine schedule, warmup.
   - Training epochs: short 5-8 epochs for baseline with early stopping (patience 2).
   - Stratified KFold (5 folds), deterministic seed; Amp + gradient accumulation if needed.
   - Balanced sampler or class weights for imbalance.

2) Validation rigor:
   - Single fixed StratifiedKFold(5, shuffle=True, random_state=42).
   - Use same preproc inside each fold; no leakage.
   - Save OOF predictions and per-fold metrics; analyze confusion matrix and per-class recall.

3) Inference:
   - EMA model or best ckpt per fold.
   - TTA (horizontal flip and 3-scale crop or resize): start with 2-4 TTAs; ensure speed.
   - Blend folds by averaging softmax logits.

4) Iteration path:
   - Baseline A: effnet_b3 380px, 5-fold, 5 epochs -> sanity OOF and first submission.
   - Baseline B: convnext_base 448/512px, 5-8 epochs -> should reach medal zone.
   - Tune aug and loss; test CutMix/MixUp vs none.
   - Train second diverse model (e.g., tf_efficientnetv2_s or tf_efficientnet_b4) and blend.
   - If time: add SWA or EMA, calibrate thresholds (not needed for accuracy).

5) Efficiency:
   - Verify GPU usable; use num_workers and prefetch; cache resized images if helpful.
   - Log epoch times, fold index, memory usage.

6) Risk checks:
   - Ensure label mapping consistent, submission format matches sample.
   - Handle EXIF orientation.
   - Deterministic seeds; track experiments.

Questions for experts (next step):
- Which backbones and input sizes typically medal on Cassava? convnext_base 512 vs. effnet_b4 512 vs. nfnet_l0 448?
- Best augmentation recipes proven on this dataset? (color jitter strength, CutMix/MixUp probs, RandAugment?)
- Preferred TTA set and count for optimal LB vs. speed?
- Any pitfalls with this dataset (duplicates, leakage, domain shifts) and best CV protocol?

Next actions:
- Verify GPU, packages (timm, albumentations, torch).
- Create training notebook: dataset, transforms, model, 5-fold loop, OOF, and inference + submission.
- Run quick smoke (1 epoch, 1 fold) to validate pipeline, then scale to 5 folds.

In [1]:
# Setup: install deps, verify GPU, quick data peek
import sys, subprocess, json, os, time, glob, platform
from pathlib import Path

def pip_install(packages):
    cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade'] + packages
    print('Installing:', ' '.join(packages)); sys.stdout.flush()
    return subprocess.run(cmd, check=True)

def pip_uninstall(packages):
    cmd = [sys.executable, '-m', 'pip', 'uninstall', '-y'] + packages
    print('Uninstalling:', ' '.join(packages)); sys.stdout.flush()
    return subprocess.run(cmd, check=False)

# Install PyTorch (CUDA 12.1) + libs if missing
try:
    import torch
    import torchvision
except Exception as e:
    print('Installing torch/torchvision for CUDA 12.1...')
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--index-url', 'https://download.pytorch.org/whl/cu121', 'torch', 'torchvision'], check=True)
    import torch, torchvision

try:
    import timm
except:
    pip_install(['timm'])
    import timm

# Prefer headless OpenCV to avoid libGL issues
cv2_ready = False
try:
    import albumentations as A
    import cv2
    cv2_ready = True
except Exception as e:
    print('Albumentations/cv2 import failed, switching to headless OpenCV. Error:', e)
    pip_uninstall(['opencv-python'])
    pip_install(['albumentations', 'opencv-python-headless'])
    import albumentations as A
    import cv2
    cv2_ready = True

import pandas as pd, numpy as np

print('Python:', platform.python_version())
print('Torch:', torch.__version__)
print('Torchvision:', torchvision.__version__)
print('timm:', timm.__version__)
print('Albumentations:', A.__version__)
print('cv2 headless OK:', cv2_ready)

# GPU check
print('GPU Available:', torch.cuda.is_available())
print('GPU Count:', torch.cuda.device_count())
if torch.cuda.is_available():
    print('GPU Name:', torch.cuda.get_device_name(0))
    props = torch.cuda.get_device_properties(0)
    print(f'GPU Memory: {props.total_memory / 1024**3:.1f} GB')

BASE = Path('.')
data_dir = BASE
train_csv = data_dir/'train.csv'
test_dir = data_dir/'test_images'
train_dir = data_dir/'train_images'
label_map_path = data_dir/'label_num_to_disease_map.json'

print('Files present:', os.listdir(data_dir))
df = pd.read_csv(train_csv)
print('train.csv shape:', df.shape)
print(df.head())
print('label value_counts:\n', df['label'].value_counts().sort_index())

with open(label_map_path) as f:
    label_map = json.load(f)
print('Label map keys:', list(label_map.keys()))

test_images = sorted([p.name for p in Path(test_dir).glob('*.jpg')])
train_images = sorted([p.name for p in Path(train_dir).glob('*.jpg')])
print('Train images:', len(train_images), 'Test images:', len(test_images))
print('Sample train images:', train_images[:5])
print('Sample test images:', test_images[:5])

# Sanity: sample_submission format
ss = pd.read_csv(data_dir/'sample_submission.csv')
print('sample_submission columns:', ss.columns.tolist(), 'shape:', ss.shape)

print('Setup complete.')

Python: 3.11.0rc1
Torch: 2.5.1+cu121
Torchvision: 0.20.1+cu121
timm: 1.0.19
Albumentations: 2.0.8
cv2 headless OK: True
GPU Available: True
GPU Count: 1
GPU Name: NVIDIA A10-24Q
GPU Memory: 23.7 GB
Files present: ['sample_submission.csv', 'test_tfrecords', 'docker_run.log', 'requirements.txt', 'test_images', 'train_with_groups.csv', 'submission.csv', 'train_images', 'train.csv', 'agent_metadata', 'task.txt', '00_eda_and_planning.ipynb', 'label_num_to_disease_map.json', 'train_tfrecords', 'description.md']
train.csv shape: (18721, 2)
         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3
label value_counts:
 label
0      939
1     1901
2     2091
3    11523
4     2267
Name: count, dtype: int64
Label map keys: ['0', '1', '2', '3', '4']
Train images: 18721 Test images: 2676
Sample train images: ['1000015157.jpg', '1000201771.jpg', '100042118.jpg', '1000723321.jpg', '1000812911.jpg']
Sample test 

In [2]:
# Dataset, transforms, and utilities
import math, random
from PIL import Image, ImageOps
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

def seed_everything(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_train_transforms(size=512):
    return A.Compose([
        A.RandomResizedCrop(size=(size, size), scale=(0.7, 1.0), ratio=(0.9, 1.1), p=1.0),
        A.HorizontalFlip(p=0.5),
        A.Affine(
            scale=(0.9, 1.1), translate_percent=(-0.1, 0.1), rotate=(-15, 15), shear=(-5, 5),
            border_mode=cv2.BORDER_REFLECT_101, value=0, p=0.7
        ),
        A.HueSaturationValue(10, 15, 10, p=0.5),
        A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
        A.GaussianBlur(blur_limit=(3, 5), p=0.1),
        A.CoarseDropout(
            max_holes=1,
            max_height=int(0.2*size), max_width=int(0.2*size),
            min_height=int(0.05*size), min_width=int(0.05*size),
            fill_value=0, p=0.15
        ),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ])

def get_valid_transforms(size=512):
    return A.Compose([
        A.LongestMaxSize(max_size=size),
        A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ])

def load_image_rgb(path: str) -> Image.Image:
    img = Image.open(path).convert('RGB')
    # handle EXIF orientation
    img = ImageOps.exif_transpose(img)
    return img

class CassavaDataset(Dataset):
    def __init__(self, df, img_dir, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transforms = transforms
        self.has_label = 'label' in self.df.columns
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['image_id']
        img = load_image_rgb(str(img_path))
        img_np = np.array(img)
        if self.transforms is not None:
            img_np = self.transforms(image=img_np)['image']
        if self.has_label:
            label = int(row['label'])
            return img_np, label
        else:
            return img_np, row['image_id']

def make_loader(df, img_dir, transforms, batch_size=32, shuffle=False, num_workers=4):
    ds = CassavaDataset(df, img_dir, transforms)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True, drop_last=shuffle, persistent_workers=num_workers>0)

def check_submission_format(sub_path='submission.csv', required_cols=('image_id','label')):
    if not os.path.exists(sub_path):
        print('submission.csv not found')
        return False
    sub = pd.read_csv(sub_path)
    ok = list(sub.columns)==list(required_cols)
    labs_ok = sub['label'].dtype.kind in 'iu' and sub['label'].between(0,4).all()
    print('Submission cols OK:', ok, 'Labels int[0..4]:', labs_ok, 'Shape:', sub.shape)
    print('Label value_counts:', sub['label'].value_counts().to_dict())
    return ok and labs_ok

seed_everything(42)
print('Utils ready.')

Utils ready.


In [3]:
# Training scaffold: model, loop, CV, and inference helpers (not executed yet)
import time
from sklearn.model_selection import StratifiedKFold
import timm
from timm.data import Mixup
from timm.utils import ModelEmaV2
from timm.loss import SoftTargetCrossEntropy

class CFG:
    seed = 42
    model_name = 'convnext_base'
    img_size = 448
    epochs = 8
    batch_size = 40
    lr = 2e-4
    weight_decay = 1e-4
    num_workers = 8
    n_splits = 5
    ls = 0.1
    mixup_alpha = 1.0
    cutmix_alpha = 1.0
    mixup_prob = 0.3
    mixup_switch_prob = 0.5
    use_mixup = True
    use_ema = True
    ema_decay = 0.999
    tta_hflip = True
    tta_scales = []  # e.g., [0.95, 1.05] later
    smoke = False  # set True for quick debug

def build_model(num_classes=5):
    model = timm.create_model(CFG.model_name, pretrained=True, num_classes=num_classes)
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    if torch.cuda.is_available():
        model = model.to(memory_format=torch.channels_last)
    return model

def get_mixup_fn():
    if not CFG.use_mixup:
        return None
    return Mixup(mixup_alpha=CFG.mixup_alpha, cutmix_alpha=CFG.cutmix_alpha, prob=CFG.mixup_prob, switch_prob=CFG.mixup_switch_prob, label_smoothing=CFG.ls, num_classes=5)

def train_one_epoch(model, loader, optimizer, scaler, mixup_fn=None, ema=None, scheduler=None):
    model.train()
    device = next(model.parameters()).device
    total_loss, total_cnt = 0.0, 0
    if mixup_fn is not None:
        criterion = SoftTargetCrossEntropy().to(device)
    else:
        criterion = nn.CrossEntropyLoss(label_smoothing=CFG.ls).to(device)
    start = time.time()
    for it, (x, y) in enumerate(loader):
        x = x.to(device, non_blocking=True)
        if torch.cuda.is_available():
            x = x.to(memory_format=torch.channels_last)
        y = y.to(device, non_blocking=True)
        if mixup_fn is not None:
            x, y = mixup_fn(x, y)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if ema is not None:
            ema.update(model)
        if scheduler is not None:
            scheduler.step()
        total_loss += loss.item() * x.size(0)
        total_cnt += x.size(0)
        if it % 50 == 0:
            elapsed = time.time() - start
            print(f'  iter {it}/{len(loader)} loss {loss.item():.4f} elapsed {elapsed:.1f}s');
            start = time.time()
    return total_loss / max(total_cnt,1)

def validate(model, loader):
    model.eval()
    device = next(model.parameters()).device
    total = 0
    correct = 0
    criterion = nn.CrossEntropyLoss().to(device)
    loss_sum = 0.0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, non_blocking=True)
            if torch.cuda.is_available():
                x = x.to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion(logits, y)
            loss_sum += loss.item() * x.size(0)
            prob = logits.softmax(dim=1)
            pred = prob.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += x.size(0)
    acc = correct / max(total,1)
    return loss_sum / max(total,1), acc

def get_scheduler(optimizer, steps_per_epoch):
    # Cosine schedule with warmup of 1 epoch
    warmup_steps = steps_per_epoch * 1
    total_steps = steps_per_epoch * CFG.epochs
    def lr_lambda(step):
        if step < warmup_steps:
            return max(1e-8, step / max(1, warmup_steps))
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
        return 0.5 * (1 + math.cos(math.pi * progress))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def run_cv_and_train(df, train_dir):
    seed_everything(CFG.seed)
    skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
    oof = np.zeros((len(df), 5), dtype=np.float32)
    fold_indices = list(skf.split(df['image_id'], df['label']))
    if CFG.smoke:
        fold_indices = fold_indices[:1]
        print('SMOKE RUN: 1 fold only, 1 epoch');
        orig_epochs = CFG.epochs; CFG.epochs = 1
    for fold, (tr_idx, va_idx) in enumerate(fold_indices):
        print(f'Fold {fold} train {len(tr_idx)} valid {len(va_idx)}')
        df_tr = df.iloc[tr_idx].reset_index(drop=True)
        df_va = df.iloc[va_idx].reset_index(drop=True)
        train_tfms = get_train_transforms(CFG.img_size)
        valid_tfms = get_valid_transforms(CFG.img_size)
        train_loader = make_loader(df_tr, train_dir, train_tfms, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
        valid_loader = make_loader(df_va, train_dir, valid_tfms, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
        model = build_model(num_classes=5)
        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        steps_per_epoch = max(1, len(train_loader))
        scheduler = get_scheduler(optimizer, steps_per_epoch)
        scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
        mixup_fn = get_mixup_fn()
        ema = ModelEmaV2(model, decay=CFG.ema_decay, device='cpu') if CFG.use_ema else None
        best_acc, best_state = -1.0, None
        for epoch in range(CFG.epochs):
            print(f'Epoch {epoch+1}/{CFG.epochs}');
            t0 = time.time()
            train_loss = train_one_epoch(model, train_loader, optimizer, scaler, mixup_fn, ema=ema, scheduler=scheduler)
            val_loss, val_acc = validate(ema.module if ema is not None else model, valid_loader)
            print(f'  train_loss {train_loss:.4f} val_loss {val_loss:.4f} val_acc {val_acc:.4f} epoch_time {time.time()-t0:.1f}s')
            if val_acc > best_acc:
                best_acc = val_acc
                best_state = (ema.module if ema is not None else model).state_dict()
        if CFG.smoke:
            CFG.epochs = orig_epochs
    return oof

def infer_test(model, df_test, test_dir, size=None, tta_hflip=True, tta_scales=None, batch_size=32):
    size = size or CFG.img_size
    dev = next(model.parameters()).device
    model.eval()
    logits_sum = []
    # Base transform
    def make_tfms(sz):
        return A.Compose([
            A.LongestMaxSize(max_size=sz),
            A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
            A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ToTensorV2()
        ])
    tfms_list = [(make_tfms(size), False)]
    if tta_hflip:
        tfms_list.append((make_tfms(size), True))
    if tta_scales:
        for s in tta_scales:
            sz = int(round(size * s))
            tfms_list.append((make_tfms(sz), False))
    for (tfms, do_flip) in tfms_list:
        ds = CassavaDataset(df_test[['image_id']].copy(), test_dir, transforms=tfms)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=True)
        part_logits = []
        with torch.no_grad():
            for x, ids in dl:
                if do_flip:
                    x = torch.flip(x, dims=[-1])
                x = x.to(dev, non_blocking=True)
                if torch.cuda.is_available():
                    x = x.to(memory_format=torch.channels_last)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    logits = model(x)
                part_logits.append(logits.float().cpu().numpy())
        logits_sum.append(np.concatenate(part_logits, axis=0))
    logits_mean = np.mean(logits_sum, axis=0)
    return logits_mean

print('Training scaffold ready. Configure CFG and call run_cv_and_train(df, train_dir) when ready.')

Training scaffold ready. Configure CFG and call run_cv_and_train(df, train_dir) when ready.


In [ ]:
# Smoke run: 1 fold x 1 epoch, then inference to submission.csv
from sklearn.model_selection import StratifiedKFold
import torch
import pandas as pd
import numpy as np

seed_everything(42)
CFG.model_name = 'convnext_tiny'
CFG.img_size = 384
CFG.batch_size = 32
CFG.epochs = 1
CFG.num_workers = 6
CFG.use_ema = True
CFG.use_mixup = True

print('Starting SMOKE training (1 fold, 1 epoch) ...')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=CFG.seed)
tr_idx, va_idx = list(skf.split(df['image_id'], df['label']))[0]
df_tr = df.iloc[tr_idx].reset_index(drop=True)
df_va = df.iloc[va_idx].reset_index(drop=True)
train_loader = make_loader(df_tr, train_dir, get_train_transforms(CFG.img_size), batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
valid_loader = make_loader(df_va, train_dir, get_valid_transforms(CFG.img_size), batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = build_model(num_classes=5)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
mixup_fn = get_mixup_fn()
ema = ModelEmaV2(model, decay=CFG.ema_decay) if CFG.use_ema else None

best_acc, best_state = -1.0, None
print('Epoch 1/1')
train_loss = train_one_epoch(model, train_loader, optimizer, scaler, mixup_fn)
if ema is not None:
    ema.update(model)
val_loss, val_acc = validate(ema.module if ema is not None else model, valid_loader)
print(f'  train_loss {train_loss:.4f} val_loss {val_loss:.4f} val_acc {val_acc:.4f}')
best_acc = val_acc
best_state = (ema.module if ema is not None else model).state_dict()

# Load best and run inference on test
if ema is not None:
    ema.module.load_state_dict(best_state)
    best_model = ema.module
else:
    model.load_state_dict(best_state)
    best_model = model

df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
logits = infer_test(best_model, df_test, test_dir, size=CFG.img_size, tta_hflip=True, tta_scales=None, batch_size=CFG.batch_size)
preds = logits.argmax(1).astype(int)
sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape:', sub.shape)
check_submission_format('submission.csv')

In [4]:
# Full training: phash groups + 5-fold convnext_base@448 with EMA + hflip+scale TTA; generate submission.csv
import time, sys, subprocess, os, torch
from sklearn.model_selection import StratifiedGroupKFold
from pathlib import Path

# Mitigate CUDA memory fragmentation
os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'expandable_segments:True,max_split_size_mb:128')

# Ensure imagehash installed
try:
    import imagehash
except Exception as e:
    print('Installing imagehash...'); sys.stdout.flush()
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ImageHash'], check=True)
    import imagehash

from PIL import ImageOps

def compute_phash_hex(path, hash_size=16):
    img = ImageOps.exif_transpose(Image.open(path).convert('RGB'))
    return str(imagehash.phash(img, hash_size=hash_size))

t0 = time.time()
# Fail-fast group merge: never silently recompute unless file missing
groups_path = Path('train_with_groups.csv')
if groups_path.exists():
    print('Loading groups from train_with_groups.csv'); sys.stdout.flush()
    gdf = pd.read_csv(groups_path)
    assert {'image_id','label','group'}.issubset(gdf.columns), 'Bad groups CSV'
    # Drop any pre-existing group-related cols to avoid suffix conflicts from prior runs
    dup_cols = [c for c in df.columns if c.startswith('group')]
    if dup_cols:
        print('Dropping pre-existing columns:', dup_cols); sys.stdout.flush()
        df = df.drop(columns=dup_cols)
    df = df.merge(gdf[['image_id','label','group']], on=['image_id','label'], how='left', validate='one_to_one')
    assert df['group'].notna().all(), 'Missing group after merge'
else:
    print('No saved groups; computing...'); sys.stdout.flush()
    df['phash'] = [compute_phash_hex(Path(train_dir)/iid) for iid in df['image_id']]
    df['group'] = df['phash'].str[:10]
    df[['image_id','label','group']].to_csv('train_with_groups.csv', index=False)

print('Unique groups:', df['group'].nunique(), 'elapsed:', f'{time.time()-t0:.1f}s')

# Configure training per expert advice
CFG.seed = 42
CFG.model_name = 'convnext_base'
CFG.img_size = 448
CFG.batch_size = 32  # A10-24GB fits 32 @448 with AMP
CFG.epochs = 10
CFG.lr = 2e-4
CFG.weight_decay = 1e-4
CFG.num_workers = 8
CFG.use_mixup = True
CFG.mixup_prob = 0.5
CFG.ls = 0.05
CFG.use_ema = True
CFG.ema_decay = 0.999
CFG.tta_scales = [0.95, 1.05]

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG.seed)
folds = list(sgkf.split(df['image_id'], df['label'], groups=df['group']))
print('Prepared StratifiedGroupKFold with 5 folds')

df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
test_logits_folds = []
oof_logits = np.zeros((len(df), 5), dtype=np.float32)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
for fold, (tr_idx, va_idx) in enumerate(folds):
    fold_start = time.time()
    print(f'===== Fold {fold} start: train {len(tr_idx)} valid {len(va_idx)} ====='); sys.stdout.flush()
    df_tr = df.iloc[tr_idx].reset_index(drop=True)
    df_va = df.iloc[va_idx].reset_index(drop=True)

    # Free any stray CUDA allocations before building the model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    train_loader = make_loader(df_tr, train_dir, get_train_transforms(CFG.img_size), batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
    valid_loader = make_loader(df_va, train_dir, get_valid_transforms(CFG.img_size), batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

    model = build_model(num_classes=5)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    steps_per_epoch = max(1, len(train_loader))
    scheduler = get_scheduler(optimizer, steps_per_epoch)
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
    mixup_fn = get_mixup_fn()
    ema = ModelEmaV2(model, decay=CFG.ema_decay, device='cpu') if CFG.use_ema else None

    best_acc, best_state = -1.0, None
    for epoch in range(CFG.epochs):
        ep_start = time.time()
        print(f'Fold {fold} Epoch {epoch+1}/{CFG.epochs}'); sys.stdout.flush()
        train_loss = train_one_epoch(model, train_loader, optimizer, scaler, mixup_fn, ema=ema, scheduler=scheduler)
        val_loss, val_acc = validate(ema.module if ema is not None else model, valid_loader)
        print(f'  train_loss {train_loss:.4f} val_loss {val_loss:.4f} val_acc {val_acc:.4f} epoch_time {time.time()-ep_start:.1f}s'); sys.stdout.flush()
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = (ema.module if ema is not None else model).state_dict()

    # Load best and compute OOF logits
    with torch.no_grad():
        target_model = ema.module if ema is not None else model
        target_model.load_state_dict(best_state)
        target_model.eval()
        dev = next(target_model.parameters()).device
        logits_all = []
        for x, y in valid_loader:
            x = x.to(dev, non_blocking=True)
            if torch.cuda.is_available():
                x = x.to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = target_model(x)
            logits_all.append(logits.float().cpu().numpy())
        logits_all = np.concatenate(logits_all, axis=0)
        oof_logits[va_idx] = logits_all
    print(f'Fold {fold} best_acc {best_acc:.4f} fold_time {time.time()-fold_start:.1f}s'); sys.stdout.flush()

    # Test inference for this fold
    fold_logits = infer_test(target_model, df_test, test_dir, size=CFG.img_size, tta_hflip=True, tta_scales=CFG.tta_scales, batch_size=CFG.batch_size)
    test_logits_folds.append(fold_logits)

# Average test logits across folds and save submission
test_logits_mean = np.mean(test_logits_folds, axis=0)
test_preds = test_logits_mean.argmax(1).astype(int)
submission = pd.DataFrame({'image_id': df_test['image_id'], 'label': test_preds})
submission.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape:', submission.shape)
check_submission_format('submission.csv')

# Save OOF logits for future ensembling if needed
np.save('oof_logits_convnext_base_448.npy', oof_logits)
print('Saved oof logits to oof_logits_convnext_base_448.npy')

Loading groups from train_with_groups.csv


Unique groups: 18721 elapsed: 0.0s


Prepared StratifiedGroupKFold with 5 folds
===== Fold 0 start: train 14976 valid 3745 =====


/tmp/ipykernel_1823/144154569.py:23: UserWarning: Argument(s) 'value' are not valid for transform Affine
  A.Affine(
/tmp/ipykernel_1823/144154569.py:30: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
/tmp/ipykernel_1823/144154569.py:43: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_CONSTANT, value=0),


/app/.pip-target/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


...

  iter 450/468 loss 0.3130 elapsed 28.0s


  train_loss 0.4560 val_loss 0.3708 val_acc 0.8889 epoch_time 774.8s


Fold 4 best_acc 0.8974 fold_time 8156.4s


/tmp/ipykernel_1823/1822884949.py:160: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),


Saved submission.csv with shape: (2676, 2)
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1673, 4: 327, 1: 281, 2: 257, 0: 138}
Saved oof logits to oof_logits_convnext_base_448.npy


In [ ]:
# Hard reset GPU context: save groups then SIGKILL kernel
import os, signal
if 'group' in df.columns:
    df[['image_id','label','group']].to_csv('train_with_groups.csv', index=False)
    print('Saved train_with_groups.csv')
else:
    print('No group column to save; skipping')
print('Killing kernel to hard-reset CUDA context...')
os.kill(os.getpid(), signal.SIGKILL)

In [5]:
# Save convnext_base test logits after Cell 5 completes
import numpy as np, os
if 'test_logits_mean' in globals():
    np.save('test_logits_convnext_base_448.npy', test_logits_mean)
    print('Saved test logits to test_logits_convnext_base_448.npy with shape:', test_logits_mean.shape)
else:
    print('test_logits_mean not found in globals; run after Cell 5 finished.')

Saved test logits to test_logits_convnext_base_448.npy with shape: (2676, 5)


In [6]:
# Train tf_efficientnet_b4_ns@512 with SGKF(phash) + ES + EMA; save fold ckpts and test logits
import time, os, sys, subprocess
from sklearn.model_selection import StratifiedGroupKFold
import torch
from timm.utils import ModelEmaV2
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy
import numpy as np
import pandas as pd
from pathlib import Path

# Speedup per expert advice
torch.backends.cudnn.benchmark = True

class CFG_B4:
    seed = 42
    model_name = 'tf_efficientnet_b4_ns'
    img_size = 512
    batch_size = 24
    epochs = 12
    min_epochs = 6
    patience = 2
    lr = 1e-4
    weight_decay = 1e-4
    num_workers = 8
    mixup_alpha = 1.0
    cutmix_alpha = 1.0
    mixup_prob = 0.5
    mixup_switch_prob = 0.5
    ls = 0.05
    use_mixup = True
    use_ema = True
    ema_decay = 0.999
    tta_scales = [0.95, 1.05]

def build_model_b4(num_classes=5):
    m = timm.create_model(CFG_B4.model_name, pretrained=True, num_classes=num_classes)
    m = m.to('cuda' if torch.cuda.is_available() else 'cpu')
    if torch.cuda.is_available():
        m = m.to(memory_format=torch.channels_last)
    return m

def get_mixup_fn_b4():
    if not CFG_B4.use_mixup:
        return None
    return Mixup(mixup_alpha=CFG_B4.mixup_alpha, cutmix_alpha=CFG_B4.cutmix_alpha, prob=CFG_B4.mixup_prob, switch_prob=CFG_B4.mixup_switch_prob, label_smoothing=CFG_B4.ls, num_classes=5)

def get_scheduler_b4(optimizer, steps_per_epoch):
    warmup = steps_per_epoch * 1
    total = steps_per_epoch * CFG_B4.epochs
    def lr_lambda(step):
        if step < warmup:
            return max(1e-8, step / max(1, warmup))
        prog = (step - warmup) / max(1, total - warmup)
        return 0.5 * (1 + math.cos(math.pi * prog))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def train_epoch_b4(model, loader, optimizer, scaler, mixup_fn=None, ema=None, scheduler=None):
    model.train()
    dev = next(model.parameters()).device
    crit = SoftTargetCrossEntropy().to(dev) if mixup_fn is not None else nn.CrossEntropyLoss(label_smoothing=CFG_B4.ls).to(dev)
    tot, cnt = 0.0, 0
    t0 = time.time()
    for it, (x, y) in enumerate(loader):
        x = x.to(dev, non_blocking=True)
        if torch.cuda.is_available(): x = x.to(memory_format=torch.channels_last)
        y = y.to(dev, non_blocking=True)
        if mixup_fn is not None:
            x, y = mixup_fn(x, y)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = crit(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if ema is not None: ema.update(model)
        if scheduler is not None: scheduler.step()
        tot += loss.item() * x.size(0)
        cnt += x.size(0)
        if it % 50 == 0:
            print(f'  iter {it}/{len(loader)} loss {loss.item():.4f}')
    return tot / max(cnt,1)

def validate_b4(model, loader):
    model.eval()
    dev = next(model.parameters()).device
    crit = nn.CrossEntropyLoss().to(dev)
    tot, cnt, correct = 0.0, 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(dev, non_blocking=True)
            if torch.cuda.is_available(): x = x.to(memory_format=torch.channels_last)
            y = y.to(dev, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = crit(logits, y)
            tot += loss.item() * x.size(0); cnt += x.size(0)
            correct += (logits.softmax(1).argmax(1) == y).sum().item()
    return tot / max(cnt,1), correct / max(cnt,1)

def train_b4_with_groups():
    seed_everything(CFG_B4.seed)
    groups_path = Path('train_with_groups.csv')
    assert groups_path.exists(), 'train_with_groups.csv missing; run Cell 5 or precompute groups first.'
    gdf = pd.read_csv(groups_path)
    # ensure df has group merged
    base_cols = ['image_id','label']
    if 'group' not in df.columns:
        mdf = df[base_cols].merge(gdf[base_cols+['group']], on=base_cols, how='left', validate='one_to_one')
    else:
        mdf = df.copy()
    assert mdf['group'].notna().all(), 'Group merge failed'
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG_B4.seed)
    folds = list(sgkf.split(mdf['image_id'], mdf['label'], groups=mdf['group']))
    df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
    test_logits_folds = []
    oof_logits = np.zeros((len(mdf), 5), dtype=np.float32)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for fold, (tr_idx, va_idx) in enumerate(folds):
        print(f'===== B4 Fold {fold} start: train {len(tr_idx)} valid {len(va_idx)} =====')
        df_tr = mdf.iloc[tr_idx].reset_index(drop=True)
        df_va = mdf.iloc[va_idx].reset_index(drop=True)
        if torch.cuda.is_available():
            torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
        train_loader = make_loader(df_tr, train_dir, get_train_transforms(CFG_B4.img_size), batch_size=CFG_B4.batch_size, shuffle=True, num_workers=CFG_B4.num_workers)
        valid_loader = make_loader(df_va, train_dir, get_valid_transforms(CFG_B4.img_size), batch_size=CFG_B4.batch_size, shuffle=False, num_workers=CFG_B4.num_workers)
        model = build_model_b4(num_classes=5)
        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG_B4.lr, weight_decay=CFG_B4.weight_decay)
        steps_per_epoch = max(1, len(train_loader))
        scheduler = get_scheduler_b4(optimizer, steps_per_epoch)
        scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
        mixup_fn = get_mixup_fn_b4()
        ema = ModelEmaV2(model, decay=CFG_B4.ema_decay, device='cpu') if CFG_B4.use_ema else None
        best_acc, best_state = -1.0, None
        no_improve = 0
        for epoch in range(CFG_B4.epochs):
            print(f'B4 Fold {fold} Epoch {epoch+1}/{CFG_B4.epochs}')
            tr_loss = train_epoch_b4(model, train_loader, optimizer, scaler, mixup_fn, ema=ema, scheduler=scheduler)
            val_loss, val_acc = validate_b4(ema.module if ema is not None else model, valid_loader)
            print(f'  train_loss {tr_loss:.4f} val_loss {val_loss:.4f} val_acc {val_acc:.4f}')
            improved = val_acc > best_acc + 1e-6
            if improved:
                best_acc = val_acc
                best_state = (ema.module if ema is not None else model).state_dict()
                no_improve = 0
                torch.save(best_state, f'ckpt_{CFG_B4.model_name}_{CFG_B4.img_size}_fold{fold}.pth')
            else:
                no_improve += 1
            if (epoch + 1) >= CFG_B4.min_epochs and no_improve > CFG_B4.patience:
                print('  Early stopping triggered')
                break
        # OOF logits
        with torch.no_grad():
            target = ema.module if ema is not None else model
            target.load_state_dict(best_state)
            target.eval()
            dev = next(target.parameters()).device
            fold_logits = []
            for x, y in valid_loader:
                x = x.to(dev, non_blocking=True)
                if torch.cuda.is_available(): x = x.to(memory_format=torch.channels_last)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    lg = target(x)
                fold_logits.append(lg.float().cpu().numpy())
            fold_logits = np.concatenate(fold_logits, axis=0)
            oof_logits[va_idx] = fold_logits
        # Test logits for this fold
        fold_test_logits = infer_test(target, df_test, test_dir, size=CFG_B4.img_size, tta_hflip=True, tta_scales=CFG_B4.tta_scales, batch_size=CFG_B4.batch_size)
        test_logits_folds.append(fold_test_logits)
        np.save(f'test_logits_{CFG_B4.model_name}_{CFG_B4.img_size}_fold{fold}.npy', fold_test_logits)
        print(f'B4 Fold {fold} best_acc {best_acc:.4f}')
    test_logits_mean = np.mean(test_logits_folds, axis=0)
    np.save(f'test_logits_{CFG_B4.model_name}_{CFG_B4.img_size}.npy', test_logits_mean)
    np.save(f'oof_logits_{CFG_B4.model_name}_{CFG_B4.img_size}.npy', oof_logits)
    print(f'Saved test logits to test_logits_{CFG_B4.model_name}_{CFG_B4.img_size}.npy with shape {test_logits_mean.shape}')
    return test_logits_mean

print('B4 training cell ready. After Cell 5 finishes, run train_b4_with_groups() to produce test logits for ensembling.')

# Auto-start training for B4
test_logits_mean_b4 = train_b4_with_groups()

B4 training cell ready. After Cell 5 finishes, run train_b4_with_groups() to produce test logits for ensembling.


===== B4 Fold 0 start: train 14976 valid 3745 =====


/tmp/ipykernel_1823/144154569.py:23: UserWarning: Argument(s) 'value' are not valid for transform Affine
  A.Affine(
/tmp/ipykernel_1823/144154569.py:30: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
/tmp/ipykernel_1823/144154569.py:43: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_CONSTANT, value=0),
/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


B4 Fold 0 Epoch 1/12


/app/.pip-target/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


...

  iter 550/624 loss 0.6268


  iter 600/624 loss 0.9666


  train_loss 0.5509 val_loss 0.4418 val_acc 0.8691
  Early stopping triggered


/tmp/ipykernel_1823/1822884949.py:160: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),


B4 Fold 4 best_acc 0.8721
Saved test logits to test_logits_tf_efficientnet_b4_ns_512.npy with shape (2676, 5)


In [ ]:
# Ensemble utility: blend convnext and effnet logits to submission
import numpy as np, pandas as pd, shutil
from pathlib import Path

def make_df_test():
    return pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})

def blend_and_write_submission(weights=(0.5, 0.5),
                               cnx_path='test_logits_convnext_base_448.npy',
                               b4_path='test_logits_tf_efficientnet_b4_ns_512.npy',
                               out_csv='submission_blend.csv'):
    w0, w1 = weights
    assert abs(w0 + w1 - 1.0) < 1e-6, 'weights must sum to 1'
    paths = [cnx_path, b4_path]
    avail = [Path(p).exists() for p in paths]
    if not all(avail):
        missing = [p for p, ok in zip(paths, avail) if not ok]
        print('Missing logits:', missing);
        return None
    L0 = np.load(cnx_path)
    L1 = np.load(b4_path)
    assert L0.shape == L1.shape, f'Logit shapes differ: {L0.shape} vs {L1.shape}'
    logits = w0 * L0 + w1 * L1
    preds = logits.argmax(1).astype(int)
    df_test = make_df_test()
    assert len(df_test) == len(preds), f'Test length mismatch: {len(df_test)} vs {len(preds)}'
    sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
    sub.to_csv(out_csv, index=False)
    print(f'Saved {out_csv} (weights={weights}) shape:', sub.shape)
    return out_csv

print('Ensembling cell ready. After both logits .npy exist, call blend_and_write_submission().')

# Auto-blend when both logits are present; also copy to submission.csv for grading
cnx_p = 'test_logits_convnext_base_448.npy'
b4_p = 'test_logits_tf_efficientnet_b4_ns_512.npy'
if Path(cnx_p).exists() and Path(b4_p).exists():
    out = blend_and_write_submission((0.5, 0.5), cnx_path=cnx_p, b4_path=b4_p, out_csv='submission_blend.csv')
    if out is not None:
        shutil.copyfile(out, 'submission.csv')
        print('Copied blend to submission.csv')
else:
    print('Blend not run yet: waiting for both logits .npy files to exist.')

In [7]:
# OOF-based probability blend with optional temperature scaling; writes final submission.csv
import numpy as np, pandas as pd, shutil, os
from pathlib import Path
from sklearn.metrics import log_loss, accuracy_score

def softmax_np(x):
    x = x - x.max(axis=1, keepdims=True)
    ex = np.exp(x)
    return ex / ex.sum(axis=1, keepdims=True)

def nll_loss(probs, y_true):
    probs = np.clip(probs, 1e-9, 1.0)
    return log_loss(y_true, probs, labels=[0,1,2,3,4])

def temperature_scale_logits(logits, T):
    return logits / float(T)

def find_temperature_on_oof(logits, y_true, T_grid=None):
    if T_grid is None:
        T_grid = [round(t,2) for t in np.arange(0.7, 1.81, 0.1)]
    best_T, best_nll = 1.0, 1e9
    for T in T_grid:
        probs = softmax_np(temperature_scale_logits(logits, T))
        nll = nll_loss(probs, y_true)
        if nll < best_nll:
            best_nll, best_T = nll, T
    return best_T, best_nll

def grid_search_weight(P0, P1, y_true, w_grid=None):
    if w_grid is None:
        # Include endpoints 0.0 (B4-only) and 1.0 (ConvNeXt-only)
        w_grid = [round(w,2) for w in np.arange(0.0, 1.0001, 0.01)]
    best_w, best_acc = 0.5, -1
    for w in w_grid:
        P = w * P0 + (1 - w) * P1
        preds = P.argmax(1)
        acc = accuracy_score(y_true, preds)
        if acc > best_acc:
            best_acc, best_w = acc, w
    return best_w, best_acc

def run_oof_tuned_blend_and_submit(
    cnx_oof_path='oof_logits_convnext_base_448.npy',
    b4_oof_path='oof_logits_tf_efficientnet_b4_ns_512.npy',
    cnx_test_path='test_logits_convnext_base_448.npy',
    b4_test_path='test_logits_tf_efficientnet_b4_ns_512.npy',
    do_temperature=True,
    out_csv='submission_blend_optimized.csv'
):
    paths = [cnx_oof_path, b4_oof_path, cnx_test_path, b4_test_path]
    missing = [p for p in paths if not Path(p).exists()]
    if missing:
        print('Missing artifacts, cannot blend yet:', missing)
        return None
    # Load
    L0_oof = np.load(cnx_oof_path)
    L1_oof = np.load(b4_oof_path)
    L0_test = np.load(cnx_test_path)
    L1_test = np.load(b4_test_path)
    assert L0_oof.shape == L1_oof.shape, f'OOF shapes mismatch: {L0_oof.shape} vs {L1_oof.shape}'
    assert L0_test.shape == L1_test.shape, f'Test shapes mismatch: {L0_test.shape} vs {L1_test.shape}'
    # True labels from current df (aligned with training order)
    y_true = df['label'].to_numpy().astype(int)
    assert len(y_true) == L0_oof.shape[0], 'OOF rows must match df length'
    # Optional temperature scaling per model (fit on OOF NLL)
    if do_temperature:
        T0, nll0 = find_temperature_on_oof(L0_oof, y_true)
        T1, nll1 = find_temperature_on_oof(L1_oof, y_true)
        print(f'ConvNeXt T*={T0} OOF-NLL={nll0:.4f}; B4 T*={T1} OOF-NLL={nll1:.4f}')
        L0_oof, L0_test = L0_oof / T0, L0_test / T0
        L1_oof, L1_test = L1_oof / T1, L1_test / T1
    # Convert to probabilities
    P0_oof = softmax_np(L0_oof)
    P1_oof = softmax_np(L1_oof)
    # Weight search on OOF accuracy
    w_best, acc_best = grid_search_weight(P0_oof, P1_oof, y_true)
    print(f'Best OOF blend weight w (convnext) = {w_best:.2f}, OOF-acc={acc_best:.5f}')
    # Apply to test
    P0_test = softmax_np(L0_test)
    P1_test = softmax_np(L1_test)
    P_test = w_best * P0_test + (1 - w_best) * P1_test
    preds = P_test.argmax(1).astype(int)
    df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
    assert len(df_test) == len(preds), 'Test length mismatch'
    sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
    sub.to_csv(out_csv, index=False)
    shutil.copyfile(out_csv, 'submission.csv')
    print('Saved', out_csv, 'and copied to submission.csv. Shape:', sub.shape)
    return out_csv, w_best, acc_best

print('OOF-tuned blending cell ready. After B4 finishes and npys exist, call:')
print("run_oof_tuned_blend_and_submit(out_csv='submission.csv')")

OOF-tuned blending cell ready. After B4 finishes and npys exist, call:
run_oof_tuned_blend_and_submit(out_csv='submission.csv')


In [10]:
# Execute OOF-tuned blend and write submission.csv
res = run_oof_tuned_blend_and_submit(out_csv='submission_blend_optimized.csv')
print('Blend result:', res)
check_submission_format('submission.csv')

ConvNeXt T*=1.0 OOF-NLL=0.3648; B4 T*=0.8 OOF-NLL=0.4138
Best OOF blend weight w (convnext) = 0.72, OOF-acc=0.89397
Saved submission_blend_optimized.csv and copied to submission.csv. Shape: (2676, 2)
Blend result: ('submission_blend_optimized.csv', 0.72, 0.8939693392446985)
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1669, 4: 337, 1: 279, 2: 257, 0: 134}


True

In [11]:
# Try multiple ensemble options (no-temp vs temp) and convnext-only; pick best by OOF acc; write submission.csv
import numpy as np, pandas as pd, shutil
from pathlib import Path

def eval_oof_acc_from_logits(logits, y_true):
    preds = logits.argmax(1)
    return (preds == y_true).mean()

def run_ensemble_search_and_submit():
    # Load artifacts
    L0_oof = np.load('oof_logits_convnext_base_448.npy')
    L1_oof = np.load('oof_logits_tf_efficientnet_b4_ns_512.npy')
    L0_test = np.load('test_logits_convnext_base_448.npy')
    L1_test = np.load('test_logits_tf_efficientnet_b4_ns_512.npy')
    y_true = df['label'].to_numpy().astype(int)
    # Baselines: single-model OOF accuracies
    acc_cnx = eval_oof_acc_from_logits(L0_oof, y_true)
    acc_b4  = eval_oof_acc_from_logits(L1_oof, y_true)
    print(f'OOF acc single: convnext={acc_cnx:.5f}, b4={acc_b4:.5f}')
    # Option A: no temperature
    P0_oof_nt = softmax_np(L0_oof); P1_oof_nt = softmax_np(L1_oof)
    wA, accA = grid_search_weight(P0_oof_nt, P1_oof_nt, y_true)
    print(f'No-temp best w={wA:.2f} OOF-acc={accA:.5f}')
    # Option B: with temperature (refit)
    T0, _ = find_temperature_on_oof(L0_oof, y_true);
    T1, _ = find_temperature_on_oof(L1_oof, y_true);
    P0_oof_t = softmax_np(L0_oof / T0); P1_oof_t = softmax_np(L1_oof / T1)
    wB, accB = grid_search_weight(P0_oof_t, P1_oof_t, y_true)
    print(f'Temp best w={wB:.2f} (T0={T0}, T1={T1}) OOF-acc={accB:.5f}')
    # Decide best option by OOF acc among: convnext-only, no-temp blend, temp blend
    choices = [
        ('convnext_only', acc_cnx, 1.0, False, 1.0, 1.0),
        ('blend_no_temp', accA, wA, False, 1.0, 1.0),
        ('blend_temp', accB, wB, True, T0, T1)
    ]
    choices.sort(key=lambda x: x[1], reverse=True)
    name_best, acc_best, w_best, use_temp, t0, t1 = choices[0]
    print(f'Chosen {name_best} with OOF-acc={acc_best:.5f}, w={w_best:.2f}, use_temp={use_temp}')
    # Build test probabilities per choice
    if name_best == 'convnext_only':
        preds = L0_test.argmax(1).astype(int)
        out_csv = 'submission_convnext_only.csv'
    else:
        if use_temp:
            P0_test = softmax_np(L0_test / t0)
            P1_test = softmax_np(L1_test / t1)
        else:
            P0_test = softmax_np(L0_test)
            P1_test = softmax_np(L1_test)
        P_test = w_best * P0_test + (1 - w_best) * P1_test
        preds = P_test.argmax(1).astype(int)
        out_csv = f'submission_{name_best}.csv'
    df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
    sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
    sub.to_csv(out_csv, index=False)
    shutil.copyfile(out_csv, 'submission.csv')
    print('Wrote', out_csv, 'and copied to submission.csv')
    check_submission_format('submission.csv')
    return {'choice': name_best, 'oof_acc': float(acc_best), 'w': float(w_best), 'use_temp': use_temp, 'T0': float(t0), 'T1': float(t1)}

print('Ensemble search cell ready. Call run_ensemble_search_and_submit() to regenerate submission.csv with the best OOF option.')

Ensemble search cell ready. Call run_ensemble_search_and_submit() to regenerate submission.csv with the best OOF option.


In [12]:
# Run ensemble search and regenerate submission.csv with best OOF option
res_search = run_ensemble_search_and_submit()
print('Search result:', res_search)

OOF acc single: convnext=0.89125, b4=0.86790
No-temp best w=0.66 OOF-acc=0.89386
Temp best w=0.72 (T0=1.0, T1=0.8) OOF-acc=0.89397
Chosen blend_temp with OOF-acc=0.89397, w=0.72, use_temp=True
Wrote submission_blend_temp.csv and copied to submission.csv
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1669, 4: 337, 1: 279, 2: 257, 0: 134}
Search result: {'choice': 'blend_temp', 'oof_acc': 0.8939693392446985, 'w': 0.72, 'use_temp': True, 'T0': 1.0, 'T1': 0.8}


In [13]:
# Generate stronger B4 test logits with extended TTA (base, hflip, vflip, rot±10) across 5 folds
import numpy as np, torch, timm, os, math, time
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

def build_b4_for_infer(num_classes=5):
    m = timm.create_model('tf_efficientnet_b4_ns', pretrained=False, num_classes=num_classes)
    m = m.to('cuda' if torch.cuda.is_available() else 'cpu')
    if torch.cuda.is_available():
        m = m.to(memory_format=torch.channels_last)
    m.eval()
    return m

def make_infer_tfms(sz=512, angle=0):
    # Rotate around center before pad/normalize
    return A.Compose([
        A.LongestMaxSize(max_size=sz),
        A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0) if angle != 0 else A.NoOp(),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2()
    ])

def infer_model_with_tta(model, df_test, img_dir, batch_size=32, size=512):
    device = next(model.parameters()).device
    # TTA plan: (angle, hflip, vflip) - add -10 with hflip for symmetry
    tta_specs = [
        (0, False, False),
        (0, True,  False),
        (0, False, True),
        (+10, False, False),
        (-10, False, False),
        (+10, True,  False),
        (-10, True,  False),
    ]
    logits_accum = None
    for (ang, hf, vf) in tta_specs:
        tfms = make_infer_tfms(size, angle=ang)
        ds = CassavaDataset(df_test[['image_id']].copy(), img_dir, transforms=tfms)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=CFG_B4.num_workers, pin_memory=True)
        part_logits = []
        with torch.no_grad():
            for x, ids in dl:
                if hf:
                    x = torch.flip(x, dims=[-1])
                if vf:
                    x = torch.flip(x, dims=[-2])
                x = x.to(device, non_blocking=True)
                if torch.cuda.is_available():
                    x = x.to(memory_format=torch.channels_last)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    lg = model(x)
                part_logits.append(lg.float().cpu().numpy())
        logits_tta = np.concatenate(part_logits, axis=0)
        if logits_accum is None:
            logits_accum = logits_tta
        else:
            logits_accum += logits_tta
    logits_mean = logits_accum / len(tta_specs)
    return logits_mean

def build_df_test():
    return pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})

def compute_b4_test_logits_extended_tta():
    df_test_local = build_df_test()
    fold_paths = [f'ckpt_tf_efficientnet_b4_ns_512_fold{i}.pth' for i in range(5)]
    for p in fold_paths:
        assert Path(p).exists(), f'Missing {p}'
    fold_logits = []
    start_all = time.time()
    for i, ckpt_path in enumerate(fold_paths):
        t0 = time.time()
        model = build_b4_for_infer(num_classes=5)
        state = torch.load(ckpt_path, map_location='cpu')
        model.load_state_dict(state, strict=True)
        lg = infer_model_with_tta(model, df_test_local, test_dir, batch_size=CFG_B4.batch_size, size=CFG_B4.img_size)
        fold_logits.append(lg)
        print(f'Fold {i} TTA inference done. Shape {lg.shape}. Elapsed {time.time()-t0:.1f}s')
        del model; torch.cuda.empty_cache()
    logits_mean = np.mean(fold_logits, axis=0)
    np.save('test_logits_tf_efficientnet_b4_ns_512_tta7.npy', logits_mean)
    # Also overwrite the default path so downstream blend uses improved logits without code changes
    np.save('test_logits_tf_efficientnet_b4_ns_512.npy', logits_mean)
    print('Saved enhanced B4 test logits to test_logits_tf_efficientnet_b4_ns_512_tta7.npy and updated default .npy. Total elapsed', f'{time.time()-start_all:.1f}s')
    return logits_mean

print('Extended TTA cell ready. Call compute_b4_test_logits_extended_tta() to refresh B4 test logits and then re-run the ensemble cell.')

Extended TTA cell ready. Call compute_b4_test_logits_extended_tta() to refresh B4 test logits and then re-run the ensemble cell.


In [17]:
# Run extended B4 TTA inference, then re-run ensemble search and regenerate submission.csv
res_tta = compute_b4_test_logits_extended_tta()
print('Enhanced B4 test logits shape:', res_tta.shape)
res_search2 = run_ensemble_search_and_submit()
print('Updated search result:', res_search2)

/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_1823/2396798903.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location='cpu')
/tmp/ipykernel_1823/2396798903.py:20: U

/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),


/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),


/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
/tmp/ipykernel_1823/2396798903.py:21: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0) if angle != 0 else A.NoOp(),


...

/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
/tmp/ipykernel_1823/2396798903.py:21: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0) if angle != 0 else A.NoOp(),


/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
/tmp/ipykernel_1823/2396798903.py:21: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0) if angle != 0 else A.NoOp(),


/tmp/ipykernel_1823/2396798903.py:20: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=sz, min_width=sz, border_mode=cv2.BORDER_CONSTANT, value=0),
/tmp/ipykernel_1823/2396798903.py:21: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0) if angle != 0 else A.NoOp(),


Fold 4 TTA inference done. Shape (2676, 5). Elapsed 63.5s
Saved enhanced B4 test logits to test_logits_tf_efficientnet_b4_ns_512_tta6.npy and updated default .npy. Total elapsed 316.7s
Enhanced B4 test logits shape: (2676, 5)
OOF acc single: convnext=0.89125, b4=0.86790
No-temp best w=0.66 OOF-acc=0.89386


Temp best w=0.72 (T0=1.0, T1=0.8) OOF-acc=0.89397
Chosen blend_temp with OOF-acc=0.89397, w=0.72, use_temp=True
Wrote submission_blend_temp.csv and copied to submission.csv
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1675, 4: 337, 1: 279, 2: 250, 0: 135}
Updated search result: {'choice': 'blend_temp', 'oof_acc': 0.8939693392446985, 'w': 0.72, 'use_temp': True, 'T0': 1.0, 'T1': 0.8}


In [15]:
# Stacking: multinomial logistic regression on concatenated OOF logits; apply to test
import numpy as np, pandas as pd, shutil
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def stack_lr_on_oof_and_submit(
    cnx_oof='oof_logits_convnext_base_448.npy',
    b4_oof='oof_logits_tf_efficientnet_b4_ns_512.npy',
    cnx_test='test_logits_convnext_base_448.npy',
    b4_test='test_logits_tf_efficientnet_b4_ns_512.npy',
    C_grid=(0.2, 0.5, 1.0),
    max_iter=300,
    out_csv='submission_stack_lr.csv'
):
    if not (Path(cnx_oof).exists() and Path(b4_oof).exists() and Path(cnx_test).exists() and Path(b4_test).exists()):
        print('Missing npy artifacts for stacking.'); return None
    L0_oof = np.load(cnx_oof); L1_oof = np.load(b4_oof)
    L0_test = np.load(cnx_test); L1_test = np.load(b4_test)
    assert L0_oof.shape == L1_oof.shape and L0_test.shape == L1_test.shape, 'Shape mismatch'
    y = df['label'].to_numpy().astype(int)
    X_oof = np.hstack([L0_oof, L1_oof])
    X_test = np.hstack([L0_test, L1_test])
    # Standardize features
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_oof_std = scaler.fit_transform(X_oof)
    X_test_std = scaler.transform(X_test)
    # CV to pick C
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_C, best_cv = None, -1.0
    for C in C_grid:
        accs = []
        for tr_idx, va_idx in skf.split(X_oof_std, y):
            Xtr, Xva = X_oof_std[tr_idx], X_oof_std[va_idx]
            ytr, yva = y[tr_idx], y[va_idx]
            clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=C, max_iter=max_iter, n_jobs=None)
            clf.fit(Xtr, ytr)
            pva = clf.predict(Xva)
            accs.append(accuracy_score(yva, pva))
        cv_mean = float(np.mean(accs))
        print(f'C={C} CV-acc={cv_mean:.5f} fold-accs={accs}')
        if cv_mean > best_cv:
            best_cv, best_C = cv_mean, C
    print(f'Chosen C={best_C} with CV-acc={best_cv:.5f}')
    # Fit on all OOF and predict test
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=best_C, max_iter=max_iter, n_jobs=None)
    clf.fit(X_oof_std, y)
    proba_test = clf.predict_proba(X_test_std)
    preds = proba_test.argmax(1).astype(int)
    df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
    sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
    sub.to_csv(out_csv, index=False)
    shutil.copyfile(out_csv, 'submission.csv')
    print('Stacking wrote', out_csv, 'and copied to submission.csv; shape:', sub.shape)
    check_submission_format('submission.csv')
    return {'C': best_C, 'cv_acc': best_cv}

print('Stacking cell ready. Call stack_lr_on_oof_and_submit() to try LR meta-ensemble and regenerate submission.csv.')

Stacking cell ready. Call stack_lr_on_oof_and_submit() to try LR meta-ensemble and regenerate submission.csv.


In [16]:
# Run stacking LR meta-ensemble and regenerate submission.csv
res_stack = stack_lr_on_oof_and_submit()
print('Stack result:', res_stack)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


C=1.0 CV-acc=0.89376 fold-accs=[0.8881174899866489, 0.8987713675213675, 0.9022435897435898, 0.8961004273504274, 0.8835470085470085]
Chosen C=0.2 with CV-acc=0.89376


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Stacking wrote submission_stack_lr.csv and copied to submission.csv; shape: (2676, 2)
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1675, 4: 320, 1: 282, 2: 264, 0: 135}
Stack result: {'C': 0.2, 'cv_acc': 0.8937559908938415}


In [19]:
# Final temp-scaled geometric-mean vs arithmetic blend with fine weight sweep; write submission.csv
import numpy as np, pandas as pd, shutil
from pathlib import Path

def row_normalize(P):
    P = np.clip(P, 1e-12, 1.0)
    P /= P.sum(axis=1, keepdims=True)
    return P

def geo_mean_blend(P0, P1, w):
    # P_final ∝ (P0^w) * (P1^(1-w))
    return row_normalize((P0 ** w) * (P1 ** (1.0 - w)))

def acc_from_probs(P, y_true):
    return (P.argmax(1) == y_true).mean(),

def final_geo_arith_blend_and_submit(
    cnx_oof='oof_logits_convnext_base_448.npy',
    b4_oof='oof_logits_tf_efficientnet_b4_ns_512.npy',
    cnx_test='test_logits_convnext_base_448.npy',
    b4_test='test_logits_tf_efficientnet_b4_ns_512.npy',
    T0=1.0, T1=0.8,
    w_lo=0.70, w_hi=0.75, w_step=0.005,
    fallback_acc=0.89397,
    out_prefix='submission_final_blend'
):
    paths = [cnx_oof, b4_oof, cnx_test, b4_test]
    miss = [p for p in paths if not Path(p).exists()]
    if miss:
        print('Missing artifacts:', miss); return None
    L0_oof = np.load(cnx_oof); L1_oof = np.load(b4_oof)
    L0_test = np.load(cnx_test); L1_test = np.load(b4_test)
    y = df['label'].to_numpy().astype(int)
    # Temp-scaled probabilities
    P0_oof = softmax_np(L0_oof / T0); P1_oof = softmax_np(L1_oof / T1)
    P0_test = softmax_np(L0_test / T0); P1_test = softmax_np(L1_test / T1)
    # Fine grid
    W = np.arange(w_lo, w_hi + 1e-9, w_step)
    best_geo = (-1.0, None)  # (acc, w)
    best_lin = (-1.0, None)
    for w in W:
        P_geo = geo_mean_blend(P0_oof, P1_oof, w)
        acc_geo = (P_geo.argmax(1) == y).mean()
        if acc_geo > best_geo[0]: best_geo = (float(acc_geo), float(w))
        P_lin = row_normalize(w * P0_oof + (1.0 - w) * P1_oof)
        acc_lin = (P_lin.argmax(1) == y).mean()
        if acc_lin > best_lin[0]: best_lin = (float(acc_lin), float(w))
    print(f'Geo-best: acc={best_geo[0]:.5f} w={best_geo[1]:.3f}; Lin-best: acc={best_lin[0]:.5f} w={best_lin[1]:.3f}')
    # Choose method
    acc_geo, w_geo = best_geo; acc_lin, w_lin = best_lin
    method = 'geo' if acc_geo >= acc_lin and acc_geo >= fallback_acc else ('lin' if acc_lin >= fallback_acc else ('geo' if acc_geo >= acc_lin else 'lin'))
    if method == 'geo':
        w_sel = w_geo
        # Tie-bias toward slightly lower w if extremely close
        if abs(acc_geo - acc_lin) < 1e-6 and w_sel > 0.72:
            w_sel = max(w_lo, w_sel - 0.01)
        P_test = geo_mean_blend(P0_test, P1_test, w_sel)
    else:
        w_sel = w_lin
        P_test = row_normalize(w_sel * P0_test + (1.0 - w_sel) * P1_test)
    preds = P_test.argmax(1).astype(int)
    df_test = pd.DataFrame({'image_id': sorted([p.name for p in Path(test_dir).glob('*.jpg')])})
    sub = pd.DataFrame({'image_id': df_test['image_id'], 'label': preds})
    out_csv = f'{out_prefix}_{method}_w{w_sel:.3f}.csv'
    sub.to_csv(out_csv, index=False)
    shutil.copyfile(out_csv, 'submission.csv')
    print(f'Wrote {out_csv} using method={method}, w={w_sel:.3f} (OOF geo={acc_geo:.5f}, lin={acc_lin:.5f}); copied to submission.csv')
    check_submission_format('submission.csv')
    return {'method': method, 'w': w_sel, 'acc_geo': acc_geo, 'acc_lin': acc_lin, 'out_csv': out_csv}

print('Final geo/arithmetic blend cell ready. Call final_geo_arith_blend_and_submit() after updating B4 TTA logits.')

Final geo/arithmetic blend cell ready. Call final_geo_arith_blend_and_submit() after updating B4 TTA logits.


In [20]:
# Final call: run geometric vs arithmetic blend with fine weight sweep and write submission.csv
res_final = final_geo_arith_blend_and_submit()
print('Final blend result:', res_final)

Geo-best: acc=0.89349 w=0.750; Lin-best: acc=0.89402 w=0.730
Wrote submission_final_blend_lin_w0.730.csv using method=lin, w=0.730 (OOF geo=0.89349, lin=0.89402); copied to submission.csv
Submission cols OK: True Labels int[0..4]: True Shape: (2676, 2)
Label value_counts: {3: 1674, 4: 337, 1: 280, 2: 250, 0: 135}
Final blend result: {'method': 'lin', 'w': 0.73, 'acc_geo': 0.8934885956946744, 'acc_lin': 0.8940227551947011, 'out_csv': 'submission_final_blend_lin_w0.730.csv'}
